<a href="https://colab.research.google.com/github/Anisha8055/ML_Projects/blob/main/movie_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv', error_bad_lines=False)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
movies= movies.merge(credits, on='title')

In [ ]:
movies.head(1)

In [ ]:
#genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies[['movie_id', 'title', 'genres', 'overview','keywords', 'cast', 'crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
#[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
#Action, Adventure, Fantasy, Science Fiction

In [ ]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter !=3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job'] =='Director':
      L.append(i['name'])
      break
  return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:str(x).split()) 

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movies['tags']= movies['overview']+ movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
newdf=movies[['movie_id','title','tags']]
newdf['tags']=newdf['tags'].apply(lambda x:" ".join(x))

In [ ]:
newdf['tags']=newdf['tags'].apply(lambda x:x.lower()) 

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)


In [ ]:
newdf['tags']=newdf['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors= cv.fit_transform(newdf['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out(50)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity= cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
  movie_index=newdf[newdf['title']==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

  for i in movies_list:
    print((i[0]), newdf.iloc[i[0]].title)
  return

In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(newdf, open('movie.pkl','wb'))

In [ ]:
newdf['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [ ]:
pickle.dump(newdf.to_dict(), open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl','wb'))